In [1]:
import torch
import torch.nn as nn
import pickle as pk
import numpy as np
from model import LocalPredictorFullSearch
import random

In [2]:
data = dict({})
for d in range(1, 29):
    filename = '../data/UsersInTokyo2012/201211{:02d}_interp.pk'.format(d)
    print(filename)
    with open(filename, 'rb') as f:
        data[d] = pk.load(f)

../data/UsersInTokyo2012/20121101_interp.pk
../data/UsersInTokyo2012/20121102_interp.pk
../data/UsersInTokyo2012/20121103_interp.pk
../data/UsersInTokyo2012/20121104_interp.pk
../data/UsersInTokyo2012/20121105_interp.pk
../data/UsersInTokyo2012/20121106_interp.pk
../data/UsersInTokyo2012/20121107_interp.pk
../data/UsersInTokyo2012/20121108_interp.pk
../data/UsersInTokyo2012/20121109_interp.pk
../data/UsersInTokyo2012/20121110_interp.pk
../data/UsersInTokyo2012/20121111_interp.pk
../data/UsersInTokyo2012/20121112_interp.pk
../data/UsersInTokyo2012/20121113_interp.pk
../data/UsersInTokyo2012/20121114_interp.pk
../data/UsersInTokyo2012/20121115_interp.pk
../data/UsersInTokyo2012/20121116_interp.pk
../data/UsersInTokyo2012/20121117_interp.pk
../data/UsersInTokyo2012/20121118_interp.pk
../data/UsersInTokyo2012/20121119_interp.pk
../data/UsersInTokyo2012/20121120_interp.pk
../data/UsersInTokyo2012/20121121_interp.pk
../data/UsersInTokyo2012/20121122_interp.pk
../data/UsersInTokyo2012/2012112

In [3]:
uid_sets = dict({})
for d in data:
    uid_sets[d] = set(list(data[d].keys()))

In [4]:
uid_doc_set = set([])
for d in range(1, 15):
    uid_doc_set = uid_doc_set | uid_sets[d]

In [5]:
data_doc = dict({})
data_qry_train = dict({})
data_qry_test = dict({})

for d in range(1, 15):
    for uid in data[d]:
        if uid not in data_doc:
            data_doc[uid] = [data[d][uid]]
        else:
            data_doc[uid].append(data[d][uid])

for d in range(15, 22):
    for uid in data[d]:
        if uid not in data_qry_train:
            data_qry_train[uid] = [data[d][uid]]
        else:
            data_qry_train[uid].append(data[d][uid])
            
for d in range(22, 29):
    for uid in data[d]:
        if uid not in data_qry_test:
            data_qry_test[uid] = [data[d][uid]]
        else:
            data_qry_test[uid].append(data[d][uid])

In [6]:
for uid in data_doc:
    data_doc[uid] = torch.LongTensor(data_doc[uid]).cuda(1)

for uid in data_qry_train:
    data_qry_train[uid] = torch.LongTensor(data_qry_train[uid]).cuda(1)
    
for uid in data_qry_test:
    data_qry_test[uid] = torch.LongTensor(data_qry_test[uid]).cuda(1)

In [7]:
num_locs = 1600
loc_embedding_dim = 128
T = 96
num_time = T
time_embedding_dim = 32
hidden_dim = 256
latent_dim = 256

In [8]:
lr = 1e-3
momentum = 0.8

In [9]:
local_predictor = LocalPredictorFullSearch(num_locs, loc_embedding_dim, num_time, time_embedding_dim, hidden_dim, latent_dim).cuda(1)
local_predictor_server = LocalPredictorFullSearch(num_locs, loc_embedding_dim, num_time, time_embedding_dim, hidden_dim, latent_dim).cuda(1)
optimizer = torch.optim.SGD(local_predictor.parameters(), lr=lr)
optimizer.zero_grad()

In [10]:
dT = 4

In [11]:
training_loss = dict({})
validation_loss = dict({})

In [12]:
user_list_train = list(data_qry_train.keys())
time_list = list(range(T - 2 * dT + 1))

# Initialize global SGD
local_predictor_update = dict({})
        
for epoch in range(1, 4001):
    
    if epoch % 500 == 0:
        lr *= 0.5
    
    update_user_list = random.sample(user_list_train, 2048)
    random.shuffle(update_user_list)
    
    avg_loss = 0.0
    cnt = 0
    n = 0
    
    uidx = 0
    
    for uid in update_user_list:
        uidx += 1
        # loading server model to local
        local_predictor.load_state_dict(local_predictor_server.state_dict())

        nk = data_qry_train[uid].shape[0]
        n += data_qry_train[uid].shape[0]

        t = np.random.randint(T - 2 * dT + 1)
        x_loc_qry = data_qry_train[uid][:, t: t + dT]
        x_t_qry = torch.zeros_like(x_loc_qry) + t
        y = data_qry_train[uid][:, t + 2 * dT - 1]

        if uid not in data_doc:
            loss = local_predictor(x_loc_qry, x_t_qry, None, None, y)
        else:
            x_loc_doc = data_doc[uid][:, t: t + 2 * dT]
            x_t_doc = torch.zeros_like(x_loc_doc) + t
            loss = local_predictor(x_loc_qry, x_t_qry, x_loc_doc, x_t_doc, y)

        loss.backward()
        avg_loss += loss.item()
        cnt += nk
        
        with torch.no_grad():
            for name, para in local_predictor.named_parameters():
                if para.grad is None:
                    continue
                elif name not in local_predictor_update:
                    local_predictor_update[name] = (para.grad + torch.randn_like(para.grad) * 1e-2) * lr
                else:
                    local_predictor_update[name] += (para.grad + torch.randn_like(para.grad) * 1e-2) * lr
                
        optimizer.zero_grad()
        
        print('Epoch {:02d}, {:.1f}%, avg_loss = {:.4f}'.format(epoch, uidx * 100 / len(update_user_list), avg_loss / cnt), end='\r')

    # update
    with torch.no_grad():
        for name, para in local_predictor_server.named_parameters():
            para -= local_predictor_update[name]
            
        for name in local_predictor_update:
            local_predictor_update[name] *= momentum
    
    training_loss[epoch] = avg_loss / cnt
    print('')
    # testing
    
    if epoch % 5 == 0:
        with torch.no_grad():

            cnt = 0
            avg_loss = 0.0

            for uid in data_qry_test:

                if np.random.ranf() > 0.05:
                    continue

                nk = data_qry_test[uid].shape[0]

                t = np.random.randint(T - 2 * dT + 1)
                x_loc_qry = data_qry_test[uid][:, t: t + dT]
                x_t_qry = torch.zeros_like(x_loc_qry) + t
                y = data_qry_test[uid][:, t + 2 * dT - 1]

                if uid not in data_doc:
                    loss = local_predictor_server(x_loc_qry, x_t_qry, None, None, y)
                else:
                    x_loc_doc = data_doc[uid][:, t: t + 2 * dT]
                    x_t_doc = torch.zeros_like(x_loc_doc) + t
                    loss = local_predictor_server(x_loc_qry, x_t_qry, x_loc_doc, x_t_doc, y)

                avg_loss += loss.item()
                cnt += nk

            print('Validation Loss: {:.4f}'.format(avg_loss / cnt))
            validation_loss[epoch] = avg_loss / cnt

Epoch 01, 100.0%, avg_loss = 7.3783
Epoch 02, 100.0%, avg_loss = 7.2986
Epoch 03, 100.0%, avg_loss = 7.1357
Epoch 04, 100.0%, avg_loss = 6.8730
Epoch 05, 100.0%, avg_loss = 6.5930
Validation Loss: 6.4149
Epoch 06, 100.0%, avg_loss = 6.3554
Epoch 07, 100.0%, avg_loss = 6.1058
Epoch 08, 100.0%, avg_loss = 5.5671
Epoch 09, 100.0%, avg_loss = 5.0436
Epoch 10, 100.0%, avg_loss = 4.5968
Validation Loss: 4.1795
Epoch 11, 100.0%, avg_loss = 4.0858
Epoch 12, 100.0%, avg_loss = 3.8204
Epoch 13, 100.0%, avg_loss = 3.5401
Epoch 14, 100.0%, avg_loss = 3.2754
Epoch 15, 100.0%, avg_loss = 3.2098
Validation Loss: 3.1315
Epoch 16, 100.0%, avg_loss = 3.1756
Epoch 17, 100.0%, avg_loss = 2.9148
Epoch 18, 100.0%, avg_loss = 2.9282
Epoch 19, 100.0%, avg_loss = 2.7448
Epoch 20, 100.0%, avg_loss = 2.6724
Validation Loss: 2.8416
Epoch 21, 100.0%, avg_loss = 2.6849
Epoch 22, 100.0%, avg_loss = 2.7125
Epoch 23, 100.0%, avg_loss = 2.6452
Epoch 24, 100.0%, avg_loss = 2.6694
Epoch 25, 100.0%, avg_loss = 2.7241
Vali

Epoch 199, 100.0%, avg_loss = 1.8512
Epoch 200, 100.0%, avg_loss = 1.8478
Validation Loss: 1.8419
Epoch 201, 100.0%, avg_loss = 1.8917
Epoch 202, 100.0%, avg_loss = 1.7780
Epoch 203, 100.0%, avg_loss = 1.9045
Epoch 204, 100.0%, avg_loss = 1.7961
Epoch 205, 100.0%, avg_loss = 1.8659
Validation Loss: 1.8424
Epoch 206, 100.0%, avg_loss = 1.8297
Epoch 207, 100.0%, avg_loss = 1.8113
Epoch 208, 100.0%, avg_loss = 1.8466
Epoch 209, 100.0%, avg_loss = 1.8451
Epoch 210, 100.0%, avg_loss = 1.7873
Validation Loss: 1.8711
Epoch 211, 100.0%, avg_loss = 1.7392
Epoch 212, 100.0%, avg_loss = 1.9183
Epoch 213, 100.0%, avg_loss = 1.8788
Epoch 214, 100.0%, avg_loss = 1.8319
Epoch 215, 100.0%, avg_loss = 1.8559
Validation Loss: 1.8251
Epoch 216, 100.0%, avg_loss = 1.8646
Epoch 217, 100.0%, avg_loss = 1.7497
Epoch 218, 100.0%, avg_loss = 1.9005
Epoch 219, 100.0%, avg_loss = 1.8728
Epoch 220, 100.0%, avg_loss = 1.8578
Validation Loss: 1.8100
Epoch 221, 100.0%, avg_loss = 1.8106
Epoch 222, 100.0%, avg_loss =

Epoch 395, 100.0%, avg_loss = 1.7724
Validation Loss: 1.7583
Epoch 396, 100.0%, avg_loss = 1.7392
Epoch 397, 100.0%, avg_loss = 1.7399
Epoch 398, 100.0%, avg_loss = 1.7238
Epoch 399, 100.0%, avg_loss = 1.6896
Epoch 400, 100.0%, avg_loss = 1.8216
Validation Loss: 1.7579
Epoch 401, 100.0%, avg_loss = 1.8548
Epoch 402, 100.0%, avg_loss = 1.7383
Epoch 403, 100.0%, avg_loss = 1.7208
Epoch 404, 100.0%, avg_loss = 1.7308
Epoch 405, 100.0%, avg_loss = 1.7438
Validation Loss: 1.8287
Epoch 406, 100.0%, avg_loss = 1.7026
Epoch 407, 100.0%, avg_loss = 1.7673
Epoch 408, 100.0%, avg_loss = 1.7610
Epoch 409, 100.0%, avg_loss = 1.6938
Epoch 410, 100.0%, avg_loss = 1.7889
Validation Loss: 1.7189
Epoch 411, 100.0%, avg_loss = 1.7921
Epoch 412, 100.0%, avg_loss = 1.7654
Epoch 413, 100.0%, avg_loss = 1.7461
Epoch 414, 100.0%, avg_loss = 1.7293
Epoch 415, 100.0%, avg_loss = 1.7399
Validation Loss: 1.7577
Epoch 416, 100.0%, avg_loss = 1.7353
Epoch 417, 100.0%, avg_loss = 1.8305
Epoch 418, 100.0%, avg_loss =

Epoch 591, 100.0%, avg_loss = 1.6308
Epoch 592, 100.0%, avg_loss = 1.6644
Epoch 593, 100.0%, avg_loss = 1.6521
Epoch 594, 100.0%, avg_loss = 1.5943
Epoch 595, 100.0%, avg_loss = 1.6481
Validation Loss: 1.6353
Epoch 596, 100.0%, avg_loss = 1.6854
Epoch 597, 100.0%, avg_loss = 1.6387
Epoch 598, 100.0%, avg_loss = 1.6251
Epoch 599, 100.0%, avg_loss = 1.6798
Epoch 600, 100.0%, avg_loss = 1.6114
Validation Loss: 1.6804
Epoch 601, 100.0%, avg_loss = 1.5494
Epoch 602, 100.0%, avg_loss = 1.6795
Epoch 603, 100.0%, avg_loss = 1.6820
Epoch 604, 100.0%, avg_loss = 1.6893
Epoch 605, 100.0%, avg_loss = 1.6840
Validation Loss: 1.6250
Epoch 606, 100.0%, avg_loss = 1.6258
Epoch 607, 100.0%, avg_loss = 1.6409
Epoch 608, 100.0%, avg_loss = 1.5855
Epoch 609, 100.0%, avg_loss = 1.7391
Epoch 610, 100.0%, avg_loss = 1.6698
Validation Loss: 1.6130
Epoch 611, 100.0%, avg_loss = 1.6557
Epoch 612, 100.0%, avg_loss = 1.5881
Epoch 613, 100.0%, avg_loss = 1.6084
Epoch 614, 100.0%, avg_loss = 1.6956
Epoch 615, 100.0

Epoch 787, 100.0%, avg_loss = 1.6221
Epoch 788, 100.0%, avg_loss = 1.6521
Epoch 789, 100.0%, avg_loss = 1.5819
Epoch 790, 100.0%, avg_loss = 1.5320
Validation Loss: 1.5937
Epoch 791, 100.0%, avg_loss = 1.5966
Epoch 792, 100.0%, avg_loss = 1.4986
Epoch 793, 100.0%, avg_loss = 1.6227
Epoch 794, 100.0%, avg_loss = 1.6686
Epoch 795, 100.0%, avg_loss = 1.6122
Validation Loss: 1.5988
Epoch 796, 100.0%, avg_loss = 1.6338
Epoch 797, 100.0%, avg_loss = 1.6054
Epoch 798, 100.0%, avg_loss = 1.5753
Epoch 799, 100.0%, avg_loss = 1.5119
Epoch 800, 100.0%, avg_loss = 1.7005
Validation Loss: 1.5636
Epoch 801, 100.0%, avg_loss = 1.5358
Epoch 802, 100.0%, avg_loss = 1.5947
Epoch 803, 100.0%, avg_loss = 1.6060
Epoch 804, 100.0%, avg_loss = 1.5805
Epoch 805, 100.0%, avg_loss = 1.6137
Validation Loss: 1.6420
Epoch 806, 100.0%, avg_loss = 1.4885
Epoch 807, 100.0%, avg_loss = 1.5774
Epoch 808, 100.0%, avg_loss = 1.5637
Epoch 809, 100.0%, avg_loss = 1.5724
Epoch 810, 100.0%, avg_loss = 1.6414
Validation Loss:

...

Epoch 3085, 100.0%, avg_loss = 1.4602
Validation Loss: 1.4540
Epoch 3086, 100.0%, avg_loss = 1.4626
Epoch 3087, 100.0%, avg_loss = 1.3636
Epoch 3088, 100.0%, avg_loss = 1.4825
Epoch 3089, 100.0%, avg_loss = 1.3811
Epoch 3090, 100.0%, avg_loss = 1.4500
Validation Loss: 1.4446
Epoch 3091, 100.0%, avg_loss = 1.4558
Epoch 3092, 100.0%, avg_loss = 1.4376
Epoch 3093, 100.0%, avg_loss = 1.4535
Epoch 3094, 100.0%, avg_loss = 1.5059
Epoch 3095, 100.0%, avg_loss = 1.5256
Validation Loss: 1.5126
Epoch 3096, 100.0%, avg_loss = 1.4536
Epoch 3097, 100.0%, avg_loss = 1.5158
Epoch 3098, 100.0%, avg_loss = 1.4572
Epoch 3099, 100.0%, avg_loss = 1.4394
Epoch 3100, 100.0%, avg_loss = 1.3826
Validation Loss: 1.4989
Epoch 3101, 100.0%, avg_loss = 1.5249
Epoch 3102, 100.0%, avg_loss = 1.4955
Epoch 3103, 100.0%, avg_loss = 1.4907
Epoch 3104, 100.0%, avg_loss = 1.3670
Epoch 3105, 100.0%, avg_loss = 1.4108
Validation Loss: 1.4835
Epoch 3106, 100.0%, avg_loss = 1.4569
Epoch 3107, 100.0%, avg_loss = 1.4869
Epoch 

Epoch 3276, 100.0%, avg_loss = 1.4535
Epoch 3277, 100.0%, avg_loss = 1.4335
Epoch 3278, 100.0%, avg_loss = 1.4090
Epoch 3279, 100.0%, avg_loss = 1.4018
Epoch 3280, 100.0%, avg_loss = 1.4462
Validation Loss: 1.4615
Epoch 3281, 100.0%, avg_loss = 1.4644
Epoch 3282, 100.0%, avg_loss = 1.4124
Epoch 3283, 100.0%, avg_loss = 1.4626
Epoch 3284, 100.0%, avg_loss = 1.4556
Epoch 3285, 100.0%, avg_loss = 1.4362
Validation Loss: 1.4584
Epoch 3286, 100.0%, avg_loss = 1.4670
Epoch 3287, 100.0%, avg_loss = 1.5412
Epoch 3288, 100.0%, avg_loss = 1.5164
Epoch 3289, 100.0%, avg_loss = 1.5370
Epoch 3290, 100.0%, avg_loss = 1.4809
Validation Loss: 1.4637
Epoch 3291, 100.0%, avg_loss = 1.4049
Epoch 3292, 100.0%, avg_loss = 1.4725
Epoch 3293, 100.0%, avg_loss = 1.4264
Epoch 3294, 100.0%, avg_loss = 1.4503
Epoch 3295, 100.0%, avg_loss = 1.3935
Validation Loss: 1.4712
Epoch 3296, 100.0%, avg_loss = 1.4599
Epoch 3297, 100.0%, avg_loss = 1.4707
Epoch 3298, 100.0%, avg_loss = 1.5047
Epoch 3299, 100.0%, avg_loss =

Epoch 3467, 100.0%, avg_loss = 1.4394
Epoch 3468, 100.0%, avg_loss = 1.4981
Epoch 3469, 100.0%, avg_loss = 1.4386
Epoch 3470, 100.0%, avg_loss = 1.4598
Validation Loss: 1.4598
Epoch 3471, 100.0%, avg_loss = 1.4263
Epoch 3472, 100.0%, avg_loss = 1.4750
Epoch 3473, 100.0%, avg_loss = 1.4467
Epoch 3474, 100.0%, avg_loss = 1.4427
Epoch 3475, 100.0%, avg_loss = 1.4711
Validation Loss: 1.4836
Epoch 3476, 100.0%, avg_loss = 1.4843
Epoch 3477, 100.0%, avg_loss = 1.3956
Epoch 3478, 100.0%, avg_loss = 1.4083
Epoch 3479, 100.0%, avg_loss = 1.5183
Epoch 3480, 100.0%, avg_loss = 1.5020
Validation Loss: 1.4688
Epoch 3481, 100.0%, avg_loss = 1.5222
Epoch 3482, 100.0%, avg_loss = 1.4893
Epoch 3483, 100.0%, avg_loss = 1.4456
Epoch 3484, 100.0%, avg_loss = 1.4522
Epoch 3485, 100.0%, avg_loss = 1.4631
Validation Loss: 1.4585
Epoch 3486, 100.0%, avg_loss = 1.4545
Epoch 3487, 100.0%, avg_loss = 1.4402
Epoch 3488, 100.0%, avg_loss = 1.4638
Epoch 3489, 100.0%, avg_loss = 1.4410
Epoch 3490, 100.0%, avg_loss =

Epoch 3658, 100.0%, avg_loss = 1.4016
Epoch 3659, 100.0%, avg_loss = 1.4906
Epoch 3660, 100.0%, avg_loss = 1.4270
Validation Loss: 1.4823
Epoch 3661, 100.0%, avg_loss = 1.4255
Epoch 3662, 100.0%, avg_loss = 1.5004
Epoch 3663, 100.0%, avg_loss = 1.4183
Epoch 3664, 100.0%, avg_loss = 1.4277
Epoch 3665, 100.0%, avg_loss = 1.4699
Validation Loss: 1.4579
Epoch 3666, 100.0%, avg_loss = 1.3990
Epoch 3667, 100.0%, avg_loss = 1.4608
Epoch 3668, 100.0%, avg_loss = 1.4466
Epoch 3669, 100.0%, avg_loss = 1.5053
Epoch 3670, 100.0%, avg_loss = 1.3599
Validation Loss: 1.4587
Epoch 3671, 100.0%, avg_loss = 1.4455
Epoch 3672, 100.0%, avg_loss = 1.4049
Epoch 3673, 100.0%, avg_loss = 1.4475
Epoch 3674, 100.0%, avg_loss = 1.4269
Epoch 3675, 100.0%, avg_loss = 1.4229
Validation Loss: 1.4979
Epoch 3676, 100.0%, avg_loss = 1.4294
Epoch 3677, 100.0%, avg_loss = 1.4264
Epoch 3678, 100.0%, avg_loss = 1.4038
Epoch 3679, 100.0%, avg_loss = 1.4406
Epoch 3680, 100.0%, avg_loss = 1.5071
Validation Loss: 1.4562
Epoch 

Epoch 3849, 100.0%, avg_loss = 1.4357
Epoch 3850, 100.0%, avg_loss = 1.4581
Validation Loss: 1.4631
Epoch 3851, 100.0%, avg_loss = 1.4784
Epoch 3852, 100.0%, avg_loss = 1.4881
Epoch 3853, 100.0%, avg_loss = 1.3773
Epoch 3854, 100.0%, avg_loss = 1.4311
Epoch 3855, 100.0%, avg_loss = 1.4714
Validation Loss: 1.4698
Epoch 3856, 100.0%, avg_loss = 1.4436
Epoch 3857, 100.0%, avg_loss = 1.4968
Epoch 3858, 100.0%, avg_loss = 1.4694
Epoch 3859, 100.0%, avg_loss = 1.3975
Epoch 3860, 100.0%, avg_loss = 1.4555
Validation Loss: 1.5038
Epoch 3861, 100.0%, avg_loss = 1.4310
Epoch 3862, 100.0%, avg_loss = 1.4407
Epoch 3863, 100.0%, avg_loss = 1.4466
Epoch 3864, 100.0%, avg_loss = 1.4191
Epoch 3865, 100.0%, avg_loss = 1.4030
Validation Loss: 1.4570
Epoch 3866, 100.0%, avg_loss = 1.4569
Epoch 3867, 100.0%, avg_loss = 1.4905
Epoch 3868, 100.0%, avg_loss = 1.4650
Epoch 3869, 100.0%, avg_loss = 1.4070
Epoch 3870, 100.0%, avg_loss = 1.4161
Validation Loss: 1.5067
Epoch 3871, 100.0%, avg_loss = 1.4148
Epoch 

In [13]:
torch.save(local_predictor, 'local_predictor_fullsearch_fl_scratch.pytorch')

In [14]:
with open('./federated_local_predictor_fullsearch_training_loss.pk', 'wb') as f:
    pk.dump(training_loss, f)
with open('./federated_local_predictor_fullsearch_validation_loss.pk', 'wb') as f:
    pk.dump(validation_loss, f)